# Ultralytics YOLO format

## 1. Directory

```powershell
<Custom Dataset Directory>
****├── 0.jpg
├── 0.txt
└── ...
```

### Split Dataset Directory

```powershell
rico_yolo
├── test
│   ├── 0.jpg
│   ├── 0.txt
│   └── ...
├── train
│   ├── 2.jpg
│   ├── 2.txt
│   └── ...
└── val
    ├── 36.jpg
    ├── 36.txt
    └── ...
```

## 2. Annotation

```
<object-class> <x> <y> <width> <height>
```

### 예시) 2.txt

13 0.5 0.4833984375 0.7944444444444444 0.626171875
3 0.5 0.758203125 0.7944444444444444 0.0765625
21 0.7875 0.758203125 0.16111111111111112 0.065625
6 0.5 0.4451171875 0.7944444444444444 0.549609375
21 0.5 0.198046875 0.6777777777777778 0.05546875
21 0.35381944444444446 0.2568359375 0.3854166666666667 0.062109375
8 0.2013888888888889 0.358984375 0.11666666666666667 0.065625
8 0.7986111111111112 0.358984375 0.11666666666666667 0.065625

## Dataset File Format

In [ ]:
yaml_text = """
train: /tf/notebook/step1/train
val: /tf/notebook/step1/val
test: /tf/notebook/step1/test

nc:13 
names:
      0: banner
      1: 2block
      2: 3block
      3: 4block
      4: 5block
      5: 6block
      6: 7block
      7: 8block
      8: nblock
      9: typo 
      10: gallery
      11: header
      12: footer
"""
with open('/tf/notebook/step1/data.yaml', 'w') as file:
    file.write(yaml_text)

In [ ]:
### 작업 디렉토리

'''powershell
weven
├── datasets
│   └── Obejectblock
└── notebooks
    └── read_rico_dataset.ipynb

'''
# Import Libraries

In [ ]:
# Step 3 Training - Preparing  Dataset for YOLO




from pathlib import Path
import json
import functools
import itertools
import shutil
import numpy as np
import pandas as pd
from PIL import Image




## For parallel computing


In [ ]:
import concurrent.futures
num_workers = 32  # The number of threads or processes

# mt_executor = concurrent.futures.ThreadPoolExecutor(num_workers)  
# Use ThreadPoolExecutor for CPU-bound tasks.
# mp_executor = concurrent.futures.ProcessPoolExecutor(num_workers)  
# Use ProcessPoolExecutor for I/O-bound tasks.

#  dataset path settings

In [ ]:
screenshots_path = Path('../datasets/Object/~')  # 66261 '.jpg' and '.json'
annotations_path = Path('../datasets/Object/~/')  # 66261 '.png' and 'json'

image_ids = pd.Series([int(p.stem) for p in screenshots_path.iterdir() if p.suffix == '.json'], name='image_id')
image_ids = image_ids.sort_values().reset_index(drop='index')

screenshot_paths = [screenshots_path / f'{image_id}.jpg' for image_id in image_ids]
annotation_paths = [annotations_path / f'{image_id}.png' for image_id in image_ids]
view_hierarchy_json_paths = [screenshots_path / f'{image_id}.json' for image_id in image_ids]
annotation_hierarchy_json_paths = [annotations_path / f'{image_id}.json' for image_id in image_ids]





## Read hierarchy json files


In [ ]:
def get_hierarchy_from_json(json_path):
    with open(json_path) as fp:
        hierarchy = json.load(fp)
        return hierarchy

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_hierarchies = mp_executor.map(get_hierarchy_from_json, annotation_hierarchy_json_paths)
    annotation_hierarchies = list(annotation_hierarchies)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    view_hierarchies = mp_executor.map(get_hierarchy_from_json, view_hierarchy_json_paths)
    view_hierarchies = list(view_hierarchies)

# Get Image sizes (가로, 세로)


In [ ]:
def get_imsize(path):
    im = Image.open(path)
    return im.size

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    screenshot_sizes = mp_executor.map(get_imsize, screenshot_paths)
    screenshot_sizes = list(screenshot_sizes)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_sizes = mp_executor.map(get_imsize, annotation_paths)
    annotation_sizes = list(annotation_sizes)

screenshot_sizes_df = pd.DataFrame(screenshot_sizes, columns=['width', 'height'])
annotation_sizes_df = pd.DataFrame(annotation_sizes, columns=['width', 'height'])

screenshot_sizes_df.insert(0, 'image_id', image_ids)
annotation_sizes_df.insert(0, 'image_id', image_ids)
screenshot_sizes_df['ratio'] = screenshot_sizes_df['width'] / screenshot_sizes_df['height']



# Get useful information from hierarchies

## A. From view_hierarchies

### 1. root size


In [ ]:
def get_root_size(hierarchy):
    return hierarchy['activity']['root']['rel-bounds']

root_sizes = map(get_root_size, view_hierarchies)

root_sizes_df = pd.DataFrame(root_sizes, columns=['x1', 'y1', 'x2', 'y2'])
root_sizes_df.insert(0, 'image_id', image_ids)
root_sizes_df['ratio'] = root_sizes_df['x2'] / root_sizes_df['y2']






### 2. root classes


In [ ]:
def get_root_classes(hierarchy):
    root_class_name = hierarchy['activity']['root']['class']
    return root_class_name

root_class_names = list(map(get_root_classes, view_hierarchies))
pd.Series(root_class_names).value_counts()





## B. From annotaion_hierarchies



In [ ]:
def get_unique_attributes(hierarchy, attribute_name):
    uniques = set()

    recursive_function = functools.partial(get_unique_attributes, attribute_name=attribute_name)
    if type(hierarchy) is list:
        uniques_list = map(recursive_function, hierarchy)
        uniques |= functools.reduce(lambda x,y: x | y, uniques_list, set())
    
    if type(hierarchy) is dict:
        if attribute_name in hierarchy:
            uniques.add(hierarchy[attribute_name])
    
        if 'children' in hierarchy:
            uniques |= recursive_function(hierarchy['children'])
    
    return uniques



### 1. componentLabel, 2. iconClass, 3.textButtonClass

In [ ]:
all_component_labels = get_unique_attributes(annotation_hierarchies, 'componentLabel')
icon_classes = get_unique_attributes(annotation_hierarchies, 'iconClass')
text_button_classes = get_unique_attributes(annotation_hierarchies, 'textButtonClass')

all_component_labels = sorted(list(all_component_labels))
icon_classes = sorted(list(icon_classes))
text_button_classes = sorted(list(text_button_classes))


# Get bounding boxes


In [ ]:
def get_component_and_bounds(hierarchy):
    bboxes = []

    if 'componentLabel' in hierarchy:
        label = hierarchy['componentLabel']
        x1, y1, x2, y2 = hierarchy['bounds']
        bboxes.append(dict(label=label, x1=x1, y1=y1, x2=x2, y2=y2))

    if 'children' in hierarchy:
        bboxes += list(itertools.chain.from_iterable(map(get_component_and_bounds, hierarchy['children'])))
    
    return bboxes

all_bboxes = list(map(get_component_and_bounds, annotation_hierarchies))


# Preprocess dataset

### Drop duplicated boxes



In [ ]:

all_bbox_dfs = list(map(lambda x: pd.DataFrame(x).drop_duplicates().reset_index(drop=True), all_bboxes))



all_bbox_df = pd.concat(all_bbox_dfs, keys=image_ids, names=['image_id', 'bbox_id'])
all_bbox_df = all_bbox_df.convert_dtypes()

### **Find landscape view (wrong matches)**


In [ ]:
landscape_screenshot_df = screenshot_sizes_df[screenshot_sizes_df['ratio'] > 1]
landscaped = all_bbox_df.index.isin(landscape_screenshot_df['image_id'], level='image_id')



### **Find overflowed boxes**


In [ ]:

annotation_sizes = {'width': 1440, 'height': 2560}
overflowed = all_bbox_df['x1'] < 0
overflowed |= all_bbox_df['y1'] < 0
overflowed |= all_bbox_df['x2'] > annotation_sizes['width']
overflowed |= all_bbox_df['y2'] > annotation_sizes['height']

### **Find boxes with 0 or negative values**

In [ ]:

negated = all_bbox_df['x1'] >= all_bbox_df['x2']
negated |= all_bbox_df['y1'] >= all_bbox_df['y2']


### **Filter out all corrupted boxes**

In [ ]:
bbox_df = all_bbox_df[~(landscaped | overflowed | negated)]

# **Convert to YOLO format**

In [ ]:
class_index = pd.Series(list(all_component_labels))
class_index = pd.Series(class_index.index.values, index=class_index)



yolo_bbox_df = pd.DataFrame({
    'label': pd.Series(class_index[bbox_df['label']].values, index=bbox_df.index),
    'x': (bbox_df['x1'] + bbox_df['x2']) / 2 / annotation_sizes['width'],
    'y': (bbox_df['y1'] + bbox_df['y2']) / 2 / annotation_sizes['height'],
    'width': (bbox_df['x2'] - bbox_df['x1']) / annotation_sizes['width'],
    'height': (bbox_df['y2'] - bbox_df['y1']) / annotation_sizes['height'],
})


# Create YOLO dataset

In [ ]:
object_yolo_path = Path('tf/datasets/datasets')
object_yolo_path.mkdir(parents=True, exist_ok=True)

### Create txt file for each image


In [ ]:
for image_id, image_bboxes_df in yolo_bbox_df.groupby('image_id'):
    image_bboxes_df.to_csv(object_yolo_path / f'{image_id}.txt', sep=' ', header=False, index=False)

### Create dataset yaml file

In [ ]:
!

In [ ]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/Object_Detect/datasets'
train_path = 'train'
test_path = 'test'
valid_path = 'valid'

# train, test, valid 폴더 생성
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)

# 파일 리스트를 무작위로 섞음
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.7 * len(file_list))
test_size = int(0.15 * len(file_list))



In [7]:
import os

# 폴더 경로 설정
folder_path = '/tf/notebook/step2_1_imagedata' # 여기에 실제 폴더 경로를 입력하세요.
start_numbers = ['0', '1', '2', '3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22']  # 여기에 원하는 숫자들을 리스트 형태로 입력하세요.

# 지정된 폴더 내의 모든 .txt 파일을 찾아서 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 첫 번째 문자가 리스트 안의 숫자 중 하나인 줄만 남김
        with open(file_path, 'w') as file:
            for line in lines:
                if line[0] in start_numbers:  # 첫 번째 문자 확인
                    file.write(line)

print("완료되었습니다.")

완료되었습니다.


In [2]:
import os

folder_path = '/tf/notebook/step2_1_imagedata' # 폴더 경로를 여기에 입력하세요
start_numbers = [str(i) for i in range(23)]  # 0부터 22까지의 숫자를 문자열 리스트로 변환

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as file:
            lines = []
            for line in file:
                if line.strip() and line[0] in start_numbers:
                    lines.append(line)

        with open(file_path, 'w') as file:
            file.writelines(lines)

print("처리 완료되었습니다.")


처리 완료되었습니다.


In [3]:
import os

folder_path = '/tf/notebook/step2_1_imagedata'   # 실제 폴더 경로로 변경하세요.
numbers_to_remove = ['99', '98', '97', '96']

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, 'r') as file:
            lines = file.readlines()

        with open(file_path, 'w') as file:
            for line in lines:
                if not any(line.startswith(num) for num in numbers_to_remove):
                    file.write(line)

print("지정된 줄을 삭제하는 작업이 완료되었습니다.")


지정된 줄을 삭제하는 작업이 완료되었습니다.


In [ ]:
import os

# 폴더 경로 설정
folder_path = '/tf/notebook/step2_1_imagedata'  # 여기에 실제 폴더 경로를 입력하세요.

# 숫자와 그에 대응하는 대체 문자열 설정
number_replacements = {'4': '0', '5': '1', '6': '2', '7': '3'}

# 지정된 폴더 내의 모든 .txt 파일을 찾아서 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 지정된 숫자로 시작하는 줄을 대체 문자열로 바꿈
        with open(file_path, 'w') as file:
            for line in lines:
                if line[0] in number_replacements:  # 첫 문자가 대체 대상 숫자인지 확인
                    line = number_replacements[line[0]] + line[1:]  # 첫 문자를 대체하고 나머지 줄을 그대로 유지
                file.write(line)

print("완료되었습니다.")

In [ ]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/split'
train_path = '/tf/notebook/split/train'
test_path = '/tf/notebook/split/test'
valid_path = '/tf/notebook/split/val'

# train, test, valid 폴더 생성
os.makedirs(train_path + '/images', exist_ok=True)
os.makedirs(train_path + '/labels', exist_ok=True)
os.makedirs(test_path + '/images', exist_ok=True)
os.makedirs(test_path + '/labels', exist_ok=True)
os.makedirs(valid_path + '/images', exist_ok=True)
os.makedirs(valid_path + '/labels', exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.7 * len(file_list))
test_size = int(0.15 * len(file_list))
valid_size = int(0.15 * len(file_list))
# train 데이터 복사
for filename in file_list[:train_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(train_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(train_path + '/labels', filename.replace('.jpg', '.txt')))

# test 데이터 복사
for filename in file_list[train_size:train_size+test_size]:
    if filename.endswith('.jpg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(test_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(test_path + '/labels', filename.replace('.jpg', '.txt')))

# validation 데이터 복사
for filename in file_list[train_size+test_size:]:
    if filename.endswith('.jpg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(valid_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(valid_path + '/labels', filename.replace('.jpg', '.txt')))

In [4]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/step2_1_imagedata'
train_path = '/tf/notebook/step2_1_imagedata/train'
test_path = '/tf/notebook/step2_1_imagedata/test'
valid_path = '/tf/notebook/step2_1_imagedata/val'

# train, test, valid 폴더 생성
os.makedirs(train_path + '/images', exist_ok=True)
os.makedirs(train_path + '/labels', exist_ok=True)
os.makedirs(test_path + '/images', exist_ok=True)
os.makedirs(test_path + '/labels', exist_ok=True)
os.makedirs(valid_path + '/images', exist_ok=True)
os.makedirs(valid_path + '/labels', exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)
file_list.sort()  # 파일 리스트를 정렬

random.shuffle(file_list)  # 파일 리스트를 셔플

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.8 * len(file_list))
test_size = int(0.1 * len(file_list))
valid_size = int(0.1 * len(file_list))
# train 데이터 복사
for filename in file_list[:train_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(train_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(train_path + '/labels', filename.replace('.jpeg', '.txt')))

# test 데이터 복사
for filename in file_list[train_size:train_size+test_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(test_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(test_path + '/labels', filename.replace('.jpeg', '.txt')))

# validation 데이터 복사
for filename in file_list[train_size+test_size:]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(valid_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(valid_path + '/labels', filename.replace('.jpeg', '.txt')))


In [ ]:
splits = ['train', 'val', 'test']
with open('data.yaml', 'w') as f:
    for split in splits:
        split_path = (object_yolo_path.parent / split).resolve()
        f.write(f'{split}: {split_path}\n')
    
    num_labels = len(all_component_labels)

    f.write('names:\n')
    for index, label in enumerate(all_component_labels):
        f.write(f'  {index}: {label}\n')

# Copy screenshot files to object_yolopath

In [ ]:
image_ids_to_copy = yolo_bbox_df.index.get_level_values('image_id').unique()

def copy_image(image_id):
    shutil.copy(screenshots_path / f'{image_id}.jpg', rico_yolo_path / f'{image_id}.jpg')
    return True

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    results = list(mp_executor.map(copy_image, image_ids_to_copy))

# Data Split yolo

### 데이터셋 path 설정

In [ ]:
object_yolo_path = Path('../datasets/object_yolo/all')

# Image Id get

In [ ]:
all_image_ids = map(lambda p :int(p.stem), object_yolo_path.iterdir())
all_image_ids = pd.Series(all_image_ids).unique()

# Random array create

In [ ]:
num_samples = len(all_image_ids)
seed_id = 0
np.ramdom.sedd(seed_id)
random_indices = np.random.choice(num_samples, num_samples, replace=False)

# Data Split

In [ ]:
splits = ['train', 'val', 'test']
split_ratio = [0.8, 0.1, 0.1]

## 랜덤 배열을 비율에 맞추어 나누기

In [ ]:
ind = np.array(split_ratio).cumsum() * num_samples
ind = np.floor(ind).astype(int) - 1

## split별 id값 얻어내기

In [ ]:
image_ids = {
    'train': all_image_ids[:ind[0]],
    'val': all_image_ids[ind[0]:ind[1]],
    'test': all_image_ids[ind[1]:],
}

## 각 split 폴더로 복사하기

In [ ]:
for split in splits:
    object_yolo_split_path = object_yolo_path.parent / split
    object_yolo_split_path.mkdir(exist_ok=True)
    for image_id in image_ids[split]:
        shutil.copy(object_yolo_path / f'{image_id}.jpg', object_yolo_split_path / f'{image_id}.jpg')
        shutil.copy(object_yolo_path / f'{image_id}.txt', object_yolo_split_path / f'{image_id}.txt')

# Docker 환경

```powershell
docker run -dit --name training_yolo --gpus all --ipc=host --mount type=volume,source=rico,destination=/weven/datasets --mount type=volume,source=runs,destination=/weven/runs ultralytics/ultralytics
```

- `-d` 컨테이너 백그라운드로 실행
- `-it` 터미널 연결
- `--gpus all` NVIDIA GPU 연결
- `--ipc=host` YOLO Docker 환경 추천 세팅
    
    > IPC mode에서 Shared memory 관련 세팅인데, 자세히 알 필요는 없어 보인다.
    > 
- `--mount type=volume,source=rico,destination=/weven/datasets` 데이터 관련 볼륨 마운트
- `--mount type=volume,source=runs,destination=/weven/runs` 학습 관련 볼륨 마운트
    - `runs`라는 이름의 볼륨이 없으면 자동으로 생성된다.
- `--name training_yolo` 컨테이너 이름
- `ultralytics/ultralytics` YOLOv8 이미지

# 작업 디렉토리

- `/weven/datasets` 는 `rico` 라는 이름의 Docker volume
    - RICO 데이터셋과 전처리 결과가 남아 있다.
- `/weven/runs` 는 `runs` 라는 이름의 Docker volume
    - 학습 세팅과 결과가 저장될 볼륨

```powershell
weven
├── datasets
│   ├── RICO
│   └── rico_yolo
│       ├── all
│       ├── test
│       ├── train
│       └── val
└── runs
    ├── configs
    │   └── ym_sgd_lr_5e-3_epochs_200.yaml
    └── train
        └── rico_yolo.yaml
```

## 학습 데이터셋 연결

- `/weven/runs/train` 디렉토리에 `rico_yolo.yaml` 파일을 미리 준비해 놓는다.

### rico_yolo.yaml

```yaml
train: /weven/datasets/rico_yolo/train
val: /weven/datasets/rico_yolo/val
test: /weven/datasets/rico_yolo/test
names:
  0: Advertisement
  1: Background Image
  2: Bottom Navigation
  3: Button Bar
  4: Card
  5: Checkbox
  6: Date Picker
  7: Drawer
  8: Icon
  9: Image
  10: Input
  11: List Item
  12: Map View
  13: Modal
  14: Multi-Tab
  15: Number Stepper
  16: On/Off Switch
  17: Pager Indicator
  18: Radio Button
  19: Slider
  20: Text
  21: Text Button
  22: Toolbar
  23: Video
  24: Web View
```

# YOLOv8 Training Configs

- 학습 세팅을 설정 파일 하나로 조절할 수 있어 매우 편하다.
- `/weven/runs/configs` 에 학습 세팅을 저장해둔다.

### ym_sgd_lr_5e-3_epochs_200.yaml

```yaml
task: detect
mode: train
model: yolov8m.pt
data: rico_yolo.yaml
epochs: 200
patience: 3
batch: 16
workers: 16
project: yolov8m
name: sgd_lr_5e-3_epochs_200
optimizer: SGD
lr0: 0.005
```

- 학습 결과 저장: `/weven/runs/train/{project}/{name}`

# Training

- 실행 위치: `/weven/runs/train`

In [ ]:
yolo task=detect mode=train model=yolov8m.pt data=/tf/notebook/datasets/data.yaml epochs=100 imgsz=640 batch=16

In [ ]:
from ultralytics import YOLO

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

results = model.train(data='data.yaml', 
                      epoch=100,
                     imgsz=640,
                     batch=16,
                     name='yolov8m_test')



In [ ]:
!pwd

In [ ]:
!ls

# 7/27 연장 학습. 기존 학습 가중치에 추가 학습. 상황에 따라 따로... 해야할 수 도 있겠다는.... 기존데이터를 지우고 새로운 데이터들로,...

In [ ]:
TORCH_USE_CUDA_DSA

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = YOLO('yolov8s.pt')

model.train(data='/tf/notebook/datasets/data.yaml',
            epochs=100,  
            lr0=0.0001, 
            imgsz=320, 
            batch=16,
            name='OBElements')

     24/100       1.3G      0.449     0.3276     0.8347        157        320: 100%|██████████| 391/391 [00:27<00:00, 14.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.33it/s]
                   all        770       7255      0.715      0.681      0.615      0.436

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     25/100      1.28G      0.444     0.3264     0.8353        103        320: 100%|██████████| 391/391 [00:27<00:00, 14.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.02it/s]
                   all        770       7255      0.812      0.621      0.655      0.466

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     26/100      1.28G     0.4434     0.3234     0.8334        107        320: 100%|██████████| 391/391 [00:28<00:00, 13.52it/s]
 

     43/100      1.28G     0.3969     0.2893     0.8254        126        320: 100%|██████████| 391/391 [00:27<00:00, 14.05it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.30it/s]
                   all        770       7255      0.814      0.689      0.686      0.522

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     44/100      1.29G     0.3983     0.2882     0.8256        110        320: 100%|██████████| 391/391 [00:27<00:00, 14.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.15it/s]
                   all        770       7255      0.868      0.679      0.726      0.534

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     45/100      1.29G     0.3914      0.283     0.8221        130        320: 100%|██████████| 391/391 [00:28<00:00, 13.81it/s]
 

     62/100      1.34G     0.3608     0.2611     0.8192        132        320: 100%|██████████| 391/391 [00:27<00:00, 14.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.26it/s]
                   all        770       7255       0.89      0.713      0.768      0.566

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     63/100      1.28G     0.3615     0.2612     0.8179        156        320: 100%|██████████| 391/391 [00:28<00:00, 13.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.11it/s]
                   all        770       7255       0.91      0.701      0.765      0.575

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     64/100      1.29G     0.3587     0.2605     0.8184         89        320: 100%|██████████| 391/391 [00:27<00:00, 14.06it/s]
 

     81/100       1.3G     0.3369     0.2408     0.8153        147        320: 100%|██████████| 391/391 [00:28<00:00, 13.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:05<00:00,  4.29it/s]
                   all        770       7255      0.956      0.712      0.786      0.599

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     82/100       1.3G     0.3322     0.2383     0.8124        138        320: 100%|██████████| 391/391 [00:28<00:00, 13.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  4.15it/s]
                   all        770       7255      0.946      0.695      0.766      0.596

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     83/100      1.29G     0.3326     0.2394     0.8131        148        320: 100%|██████████| 391/391 [00:27<00:00, 13.96it/s]
 

In [ ]:
import cv2
import numpy as np

def find_most_common_color(image_path):
    # 이미지 불러오기
    image = cv2.imread(image_path)

    # 이미지를 1차원 배열로 변환 (픽셀별 RGB 값)
    pixels = image.reshape(-1, 3)

    # RGB 값들의 빈도를 계산
    unique_colors, counts = np.unique(pixels, axis=0, return_counts=True)

    # 가장 빈도가 높은 RGB 값과 빈도 수 계산
    most_common_index = np.argmax(counts)
    most_common_color = unique_colors[most_common_index]
    most_common_count = counts[most_common_index]

    return most_common_color, most_common_count

if __name__ == "__main__":
    image_path = "/tf/notebook/test_004.png"
    most_common_color, most_common_count = find_most_common_color(image_path)
    print(f"The most common color in the image is: {most_common_color}, count: {most_common_count}")

In [ ]:
model.train(resume=True)

In [ ]:
model = YOLO('/tf/notebook/runs/detect/train13/weights/last.pt')

model.train(data='/tf/notebook/datasetstest/data.yaml',
            epochs=50,
            batch=32,
            imgsz=320,
            lr0=0.001 
           )

In [ ]:
model = YOLO('/tf/notebook/runs/detect/train13/weights/last.pt')
model.train(resume=True)

In [ ]:
source='/tf/notebook/test_001.png'

In [ ]:
!yolo predict model=/tf/notebook/runs/detect/train13/weights/last.pt source=/tf/notebook/test_011.png 

In [ ]:
model = YOLO('yolov8m.pt')

model.train(data='/tf/notebook/datasetsprice/data.yaml',
            epochs=200,
            batch=32,
            imgsz=480,
            lr0=0.001 
           )

     26/200       8.2G     0.7202     0.3429      0.785        612        480: 100%|██████████| 35/35 [00:07<00:00,  4.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.84it/s]
                   all        141       1729      0.978      0.929      0.972      0.798

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     27/200      8.21G     0.6724     0.3316     0.7867        652        480: 100%|██████████| 35/35 [00:07<00:00,  4.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.91it/s]
                   all        141       1729      0.988      0.931       0.97      0.793

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     28/200      8.28G     0.6493     0.3176     0.7839        565        480: 100%|██████████| 35/35 [00:07<00:00,  4.67it/s]
           

     64/200      8.29G     0.5392     0.2631     0.7746        639        480: 100%|██████████| 35/35 [00:08<00:00,  4.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.68it/s]
                   all        141       1729       0.99      0.928      0.969        0.8

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     65/200      8.24G     0.5303      0.263     0.7762        609        480: 100%|██████████| 35/35 [00:07<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.57it/s]
                   all        141       1729      0.991      0.923      0.966      0.789

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     66/200       8.2G     0.5306     0.2672     0.7762        581        480: 100%|██████████| 35/35 [00:07<00:00,  4.68it/s]
           

    102/200      8.29G     0.4698     0.2398     0.7711        650        480: 100%|██████████| 35/35 [00:07<00:00,  4.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.00it/s]
                   all        141       1729      0.994      0.912      0.963      0.796

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    103/200      8.24G     0.4841      0.242     0.7721        551        480: 100%|██████████| 35/35 [00:07<00:00,  4.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.02it/s]
                   all        141       1729      0.992      0.921      0.964      0.795

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    104/200       8.2G     0.4799     0.2393     0.7734        553        480: 100%|██████████| 35/35 [00:07<00:00,  4.70it/s]
           

    140/200      8.29G     0.4173     0.2125     0.7693        656        480: 100%|██████████| 35/35 [00:07<00:00,  4.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.85it/s]
                   all        141       1729      0.992      0.932      0.963      0.813

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    141/200      8.24G     0.4271     0.2125     0.7664        614        480: 100%|██████████| 35/35 [00:07<00:00,  4.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.01it/s]
                   all        141       1729      0.985      0.941      0.967      0.822

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    142/200      8.21G     0.4225     0.2141     0.7655        633        480: 100%|██████████| 35/35 [00:07<00:00,  4.71it/s]
           

    178/200      8.21G     0.3916     0.1923     0.7655        608        480: 100%|██████████| 35/35 [00:07<00:00,  4.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.05it/s]
                   all        141       1729      0.989      0.938      0.966       0.82

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    179/200      8.23G     0.3793     0.1876     0.7659        640        480: 100%|██████████| 35/35 [00:07<00:00,  4.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.77it/s]
                   all        141       1729      0.991      0.938      0.966      0.826

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    180/200      8.19G     0.3729     0.1889      0.762        643        480: 100%|██████████| 35/35 [00:07<00:00,  4.75it/s]
           

    197/200       8.1G     0.3879     0.1951     0.7636        312        480: 100%|██████████| 35/35 [00:07<00:00,  4.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.98it/s]
                   all        141       1729      0.993      0.924      0.962      0.917

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    198/200      8.08G     0.3836     0.1911     0.7605        311        480: 100%|██████████| 35/35 [00:07<00:00,  4.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.96it/s]
                   all        141       1729      0.993      0.923      0.962      0.921

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    199/200       8.1G     0.3771     0.1847     0.7664        360        480: 100%|██████████| 35/35 [00:07<00:00,  4.81it/s]
           

In [ ]:
!yolo predict model=/tf/notebook/runs/detect/train19/weights/last.pt source=/tf/notebook/test_008.png 

In [ ]:
model = YOLO('/tf/notebook/runs/detect/train19/weights/last.pt')

model.train(data='/tf/notebook/datasets_1/data.yaml',
           epochs=75,
           batch=16,
           imgsz=480,
           lr0=0.01)

       6/75      4.04G     0.4896     0.3826     0.8655         18        480: 100%|██████████| 461/461 [00:55<00:00,  8.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.69it/s]
                   all        911       8984      0.791      0.703      0.668       0.53

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       7/75      4.06G     0.4747      0.368     0.8605          8        480: 100%|██████████| 461/461 [00:55<00:00,  8.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.74it/s]
                   all        911       8984       0.74      0.629      0.659      0.549

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       8/75      4.05G     0.4588     0.3533     0.8566          7        480: 100%|██████████| 461/461 [00:55<00:00,  8.29it/s]
 

      25/75      4.05G     0.3656     0.2718     0.8269          2        480: 100%|██████████| 461/461 [00:55<00:00,  8.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.76it/s]
                   all        911       8984      0.863      0.886      0.888      0.714

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      26/75      4.07G     0.3639     0.2663     0.8262         22        480: 100%|██████████| 461/461 [00:56<00:00,  8.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.75it/s]
                   all        911       8984       0.91      0.773      0.817      0.682

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      27/75      4.06G     0.3553     0.2599     0.8243         18        480: 100%|██████████| 461/461 [00:55<00:00,  8.29it/s]
 

      44/75      4.06G     0.3166     0.2277     0.8136         27        480: 100%|██████████| 461/461 [00:55<00:00,  8.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.77it/s]
                   all        911       8984      0.978      0.803      0.841      0.724

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      45/75      3.98G     0.3106     0.2253     0.8142          6        480: 100%|██████████| 461/461 [00:55<00:00,  8.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.78it/s]
                   all        911       8984      0.766      0.879      0.868      0.741

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      46/75      4.05G     0.3078     0.2205     0.8132          7        480: 100%|██████████| 461/461 [00:55<00:00,  8.23it/s]
 

      63/75      4.06G     0.2757     0.1996     0.8048          9        480: 100%|██████████| 461/461 [00:56<00:00,  8.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.74it/s]
                   all        911       8984       0.77        0.9      0.871      0.754

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      64/75      4.16G     0.2735     0.1942      0.805          8        480: 100%|██████████| 461/461 [00:56<00:00,  8.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:08<00:00,  3.59it/s]
                   all        911       8984      0.977      0.803       0.91      0.787

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      65/75      4.05G     0.2714     0.1924      0.804          7        480: 100%|██████████| 461/461 [00:57<00:00,  8.06it/s]
 

In [ ]:
!yolo detect train model=/tf/notebook/runs/detect/train22/weights/best.pt data=/tf/notebook/datasets_1/data.yaml epochs=75 batch=32 imgsz=480 lr0=0.01

In [ ]:
!yolo detect train model=/tf/notebook/runs/detect/OBElements5/weights/last.pt data=/tf/notebook/datasetsprice/data.yaml epochs=100 batch=16 imgsz=640 lr0=0.005

In [ ]:
!yolo detect train model=yolov8m.pt data=/tf/notebook/datasetsprice/data.yaml epochs=100 batch=16 imgsz=480 lr0=0.005


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      8/100      3.95G     0.5899     0.4236     0.8706          7        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.656      0.715      0.629      0.481

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      9/100      3.94G     0.5704      0.414     0.8667          5        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.898      0.625       0.72      0.536

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     10/100      4.06G     0.5551      0.397     0.8608         17        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.688      0.722      0.692      0.545

      Epo


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     33/100      3.99G     0.3867     0.2744     0.8231         10        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.954      0.777      0.821      0.691

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     34/100      3.95G     0.3873     0.2753     0.8243         17        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.933      0.749      0.817      0.703

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     35/100      3.97G     0.3805     0.2684     0.8231          7        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.922       0.79      0.813      0.703

      Epo


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     58/100      3.94G     0.3188     0.2256     0.8096         23        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.784      0.883      0.856      0.735

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     59/100      3.96G      0.316     0.2238     0.8098         16        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.963      0.805      0.903      0.757

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     60/100      3.94G     0.3192     0.2255     0.8102         15        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.805      0.898      0.906      0.763

      Epo


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     83/100      3.97G     0.2705     0.1901     0.8021         22        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.831      0.887      0.905      0.777

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     84/100      3.97G     0.2718     0.1889     0.8017         31        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984      0.814      0.899      0.869      0.765

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     85/100      3.95G     0.2674     0.1892     0.8021          5        480: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        911       8984       0.82      0.898       0.87      0.766

      Epo

In [ ]:
!yolo predict model=/tf/notebook/runs/detect/train61/weights/last.pt source=/tf/notebook/test_024.png conf=0.1 iou=0.1

In [ ]:
!yolo detect train model=yolov8l.pt data=/tf/notebook/datasets/data.yaml epochs=100 batch=128 imgsz=640 lr0=0.005

Aborted


In [ ]:
from ultralytics import YOLO
import torch, gc
gc.collect()
torch.cuda.empty_cache()
model = YOLO('yolov8l.pt')

model.train(data='/tf/notebook/datasets/data.yaml', 
                    epochs=100,
                    imgsz=480,
                    batch=16,
                    lr0=0.005)



     26/100      5.85G     0.3548     0.2675      0.831         91        480: 100%|██████████| 568/568 [01:36<00:00,  5.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.91it/s]
                   all       1130       9998      0.819      0.899      0.883       0.72

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     27/100      5.79G      0.354     0.2633     0.8298         91        480: 100%|██████████| 568/568 [01:37<00:00,  5.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.87it/s]
                   all       1130       9998      0.882      0.891      0.908      0.756

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     28/100      5.83G     0.3506     0.2627     0.8275         88        480: 100%|██████████| 568/568 [01:37<00:00,  5.84it/s]
 

     45/100      5.84G     0.3016     0.2243     0.8173        104        480: 100%|██████████| 568/568 [01:35<00:00,  5.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.97it/s]
                   all       1130       9998       0.89       0.93      0.942      0.817

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     46/100      5.79G     0.3014     0.2244     0.8177         98        480: 100%|██████████| 568/568 [01:34<00:00,  6.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:08<00:00,  4.01it/s]
                   all       1130       9998      0.923      0.924      0.959      0.837

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     47/100      5.84G     0.3006     0.2237     0.8171         61        480: 100%|██████████| 568/568 [01:35<00:00,  5.97it/s]
 

     64/100      5.83G        inf     0.1972     0.8093         58        480: 100%|██████████| 568/568 [01:37<00:00,  5.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.88it/s]
                   all       1130       9998      0.954      0.929      0.958      0.858

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     65/100      5.85G     0.2662     0.1974     0.8107        112        480: 100%|██████████| 568/568 [01:37<00:00,  5.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.89it/s]
                   all       1130       9998      0.967      0.927      0.964      0.863

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     66/100      5.87G     0.2664     0.1975     0.8102         60        480: 100%|██████████| 568/568 [01:37<00:00,  5.82it/s]
 

     83/100      5.78G     0.2353     0.1741     0.8028        132        480: 100%|██████████| 568/568 [01:34<00:00,  6.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  4.00it/s]
                   all       1130       9998      0.975      0.934      0.966      0.877

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     84/100      5.84G     0.2347     0.1727     0.8032         84        480: 100%|██████████| 568/568 [01:34<00:00,  6.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.95it/s]
                   all       1130       9998      0.969      0.933      0.967      0.878

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     85/100      5.79G     0.2328     0.1703     0.8031         34        480: 100%|██████████| 568/568 [01:35<00:00,  5.96it/s]
 

In [ ]:
!yolo predict model=/tf/notebook/runs/detect/train58/weights/last.pt source=/tf/notebook/test_021.png

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPU is available")
    for device in physical_devices:
        print(device)
else:
    print("GPU is not available")


In [ ]:
!nvidia-smi

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from ultralytics import YOLO
import torch, gc
gc.collect()
torch.cuda.empty_cache()
model = YOLO('/tf/notebook/runs/detect/train61/weights/last.pt')

model.train(data='/tf/notebook/datasets/data.yaml', 
                    epochs=75,
                    imgsz=480,
                    batch=8,
                    lr0=0.001)



In [ ]:
!yolo task=detect mode=predict model=/tf/notebook/best_v8m.pt source='/tf/notebook/testimage/' save_crop

image 50/2605 /tf/notebook/testimage/screencapture-aloshop-7uptheme-net-home-14-2022-09-27-13_44_52.jpg: 640x320 1 2block, 1 3block, 5 5blocks, 1 header, 1 footer, 72.1ms
image 51/2605 /tf/notebook/testimage/screencapture-aloshop-7uptheme-net-home-15-2022-09-27-13_44_59.jpg: 640x256 1 banner, 2 2blocks, 7 4blocks, 1 typo, 6 gallerys, 1 header, 1 footer, 40.4ms
image 52/2605 /tf/notebook/testimage/screencapture-aloshop-7uptheme-net-home-17-2022-09-27-13_45_12.jpg: 640x224 3 banners, 3 2blocks, 1 4block, 6 5blocks, 2 7blocks, 1 header, 1 footer, 36.1ms
image 53/2605 /tf/notebook/testimage/screencapture-aloshop-7uptheme-net-home-19-2022-09-27-13_45_21.jpg: 640x288 1 banner, 3 2blocks, 5 3blocks, 3 7blocks, 1 typo, 1 gallery, 1 header, 1 footer, 48.9ms
image 54/2605 /tf/notebook/testimage/screencapture-aloshop-7uptheme-net-home-21-2022-09-27-13_45_28.jpg: 640x256 2 banners, 2 3blocks, 3 5blocks, 1 gallery, 1 header, 1 footer, 43.2ms
image 55/2605 /tf/notebook/testimage/screencapture-alosho

image 93/2605 /tf/notebook/testimage/screencapture-ash-us-en-2022-09-29-16_40_42.jpg: 640x320 1 banner, 1 2block, 1 4block, 1 typo, 1 gallery, 1 header, 1 footer, 26.5ms
image 94/2605 /tf/notebook/testimage/screencapture-asics-co-kr-2022-09-28-17_03_56.jpg: 640x352 2 banners, 2 2blocks, 1 3block, 1 4block, 1 header, 1 footer, 31.6ms
image 95/2605 /tf/notebook/testimage/screencapture-athletekorea-2022-09-29-11_45_59.jpg: 640x160 2 banners, 1 2block, 4 4blocks, 1 gallery, 1 header, 1 footer, 26.5ms
image 96/2605 /tf/notebook/testimage/screencapture-attrangs-co-kr-2022-09-29-11_21_28.jpg: 640x224 2 banners, 2 2blocks, 3 3blocks, 1 4block, 3 5blocks, 1 typo, 1 gallery, 29.7ms
image 97/2605 /tf/notebook/testimage/screencapture-aty022-aty-kr-1511492415854-main-2022-09-21-18_17_08.jpg: 640x320 3 banners, 1 2block, 1 3block, 5 4blocks, 1 typo, 1 gallery, 1 header, 1 footer, 57.1ms
image 98/2605 /tf/notebook/testimage/screencapture-aty043-aty-kr-1512370004336-main-2022-09-21-18_16_24.jpg: 640x2

image 138/2605 /tf/notebook/testimage/screencapture-babo-wpjavo-home-8-ladys-clothe-2022-09-27-17_56_27.jpg: 640x192 2 banners, 2 2blocks, 1 3block, 4 4blocks, 1 typo, 1 header, 1 footer, 17.9ms
image 139/2605 /tf/notebook/testimage/screencapture-babo-wpjavo-home-9-accessories-2022-09-27-17_56_37.jpg: 640x256 3 banners, 1 3block, 2 4blocks, 1 7block, 1 header, 1 footer, 20.3ms
image 140/2605 /tf/notebook/testimage/screencapture-babo-wpjavo-home-portal-1-2022-09-27-18_00_14.jpg: 640x320 1 banner, 1 2block, 2 3blocks, 1 4block, 3 5blocks, 1 typo, 2 gallerys, 1 header, 1 footer, 55.2ms
image 141/2605 /tf/notebook/testimage/screencapture-babo-wpjavo-home-portal-3-2022-09-27-18_00_31.jpg: 640x288 1 banner, 1 2block, 3 3blocks, 1 4block, 4 5blocks, 1 typo, 2 gallerys, 1 header, 1 footer, 77.1ms
image 142/2605 /tf/notebook/testimage/screencapture-babo-wpjavo-home-portal-4-2022-09-27-18_00_39.jpg: 640x320 1 banner, 1 2block, 2 3blocks, 1 4block, 3 5blocks, 1 typo, 1 gallery, 1 header, 1 footer

image 182/2605 /tf/notebook/testimage/screencapture-bestcareerbd-themes-minoan-home-six-2022-09-26-10_24_13.jpg: 640x448 1 banner, 1 5block, 1 header, 1 footer, 6.7ms
image 183/2605 /tf/notebook/testimage/screencapture-bestcareerbd-themes-minoan-home-two-2022-09-26-10_19_22.jpg: 640x320 2 banners, 1 2block, 2 3blocks, 2 4blocks, 1 5block, 1 gallery, 1 header, 1 footer, 6.7ms
image 184/2605 /tf/notebook/testimage/screencapture-bidpro-webdevia-2022-09-23-15_42_00.jpg: 640x320 1 banner, 3 3blocks, 2 4blocks, 1 6block, 1 gallery, 1 header, 1 footer, 6.2ms
image 185/2605 /tf/notebook/testimage/screencapture-bidpro-webdevia-construction-equipment-marketplace-2022-09-23-15_43_21.jpg: 640x352 1 banner, 2 3blocks, 2 4blocks, 1 6block, 1 gallery, 1 header, 1 footer, 6.6ms
image 186/2605 /tf/notebook/testimage/screencapture-bidpro-webdevia-real-estate-auction-marketplace-2022-09-23-15_42_36.jpg: 640x448 1 banner, 1 3block, 3 4blocks, 1 typo, 1 header, 1 footer, 7.4ms
image 187/2605 /tf/notebook/t

image 228/2605 /tf/notebook/testimage/screencapture-blueskytechco-homess-home-page-6-2022-09-27-17_09_32145.jpg: 640x352 1 banner, 1 3block, 1 typo, 1 gallery, 1 header, 1 footer, 111.7ms
image 229/2605 /tf/notebook/testimage/screencapture-blueskytechco-homess-home-page-7-2022-09-27-17_09_42146.jpg: 640x288 1 banner, 3 3blocks, 2 4blocks, 1 typo, 1 header, 1 footer, 27.3ms
image 230/2605 /tf/notebook/testimage/screencapture-blueskytechco-homess-home-page-8-2022-09-27-17_09_53147.jpg: 640x384 1 banner, 2 3blocks, 1 4block, 1 typo, 1 header, 1 footer, 29.6ms
image 231/2605 /tf/notebook/testimage/screencapture-blueskytechco-rubix-2022-09-27-14_01_47159.jpg: 640x224 3 banners, 2 3blocks, 2 4blocks, 2 6blocks, 1 gallery, 1 header, 1 footer, 28.3ms
image 232/2605 /tf/notebook/testimage/screencapture-blueskytechco-rubix-fashion-01-2022-09-27-14_03_54161.jpg: 640x192 1 banner, 1 2block, 2 3blocks, 2 4blocks, 2 6blocks, 1 typo, 1 gallery, 1 header, 1 footer, 22.1ms
image 233/2605 /tf/notebook/t

...

image 2421/2605 /tf/notebook/testimage/screencapture-parris-qodeinteractive-2022-09-28-13_38_05.jpg: 640x192 3 banners, 1 2block, 3 3blocks, 1 6block, 1 7block, 1 header, 1 footer, 39.9ms
image 2422/2605 /tf/notebook/testimage/screencapture-parris-qodeinteractive-cosmetics-store-2022-09-28-13_38_27.jpg: 640x224 1 banner, 2 2blocks, 1 3block, 3 4blocks, 1 7block, 1 8block, 1 typo, 1 header, 1 footer, 51.8ms
image 2423/2605 /tf/notebook/testimage/screencapture-passim-qodeinteractive-beauty-brand-2022-09-28-13_37_19.jpg: 640x192 5 banners, 1 2block, 2 3blocks, 1 4block, 1 typo, 1 gallery, 1 header, 1 footer, 20.2ms
image 2424/2605 /tf/notebook/testimage/screencapture-passion-jwsuperthemes-homepage-04-2022-09-28-10_53_351557.jpg: 640x288 1 banner, 1 3block, 1 4block, 1 6block, 2 gallerys, 1 header, 1 footer, 24.4ms
image 2425/2605 /tf/notebook/testimage/screencapture-pastelmall-2022-09-28-17_15_07.jpg: 640x192 2 banners, 2 2blocks, 1 4block, 4 5blocks, 1 6block, 1 typo, 1 gallery, 1 header

image 2464/2605 /tf/notebook/testimage/screencapture-quartz-wp1-zootemplate-home-6-2022-09-28-10_00_45.jpg: 640x640 1 banner, 1 2block, 3 4blocks, 2 headers, 1 footer, 250.2ms
image 2465/2605 /tf/notebook/testimage/screencapture-quartz-wp1-zootemplate-home-7-2022-09-28-10_00_50.jpg: 640x416 3 banners, 2 3blocks, 2 4blocks, 1 5block, 1 header, 1 footer, 7.2ms
image 2466/2605 /tf/notebook/testimage/screencapture-qvo-co-kr-2022-09-29-13_09_40.jpg: 640x192 2 banners, 2 3blocks, 2 4blocks, 3 5blocks, 2 gallerys, 1 header, 7.4ms
image 2467/2605 /tf/notebook/testimage/screencapture-rab-kaththemes-shoes-2022-09-27-13_36_421918.jpg: 640x256 2 banners, 2 3blocks, 3 4blocks, 1 typo, 1 header, 1 footer, 6.9ms
image 2468/2605 /tf/notebook/testimage/screencapture-rachelcox-co-kr-2022-09-29-14_23_25.jpg: 640x192 3 banners, 1 2block, 1 3block, 1 typo, 1 header, 1 footer, 6.6ms
image 2469/2605 /tf/notebook/testimage/screencapture-radiustheme-demo-wordpress-themes-medimall-home-2-2022-09-23-17_30_40.jpg

image 2508/2605 /tf/notebook/testimage/screencapture-sentinal-7uptheme-net-home-4-2022-09-26-15_46_32.jpg: 640x352 2 banners, 2 3blocks, 2 4blocks, 1 header, 1 footer, 21.9ms
image 2509/2605 /tf/notebook/testimage/screencapture-sentinal-7uptheme-net-home-5-2022-09-26-15_46_44.jpg: 640x256 3 banners, 2 3blocks, 3 4blocks, 1 gallery, 1 header, 1 footer, 27.1ms
image 2510/2605 /tf/notebook/testimage/screencapture-sentinal-7uptheme-net-home-8-2022-09-26-15_47_26.jpg: 640x256 2 banners, 1 2block, 2 4blocks, 2 typos, 1 gallery, 2 headers, 23.8ms
image 2511/2605 /tf/notebook/testimage/screencapture-sentinal-7uptheme-net-home-9-2022-09-26-15_47_37.jpg: 640x224 2 banners, 3 2blocks, 1 3block, 3 4blocks, 1 5block, 1 typo, 2 headers, 1 footer, 26.0ms
image 2512/2605 /tf/notebook/testimage/screencapture-sephora-kr-2022-09-29-13_42_02.jpg: 640x608 2 banners, 1 3block, 1 6block, 2 typos, 1 header, 67.3ms
image 2513/2605 /tf/notebook/testimage/screencapture-sezwick-2022-09-30-09_50_59.jpg: 640x224 1 

image 2553/2605 /tf/notebook/testimage/screencapture-styleggom-co-kr-2022-09-29-14_11_37.jpg: 640x160 1 2block, 1 4block, 6 5blocks, 2 6blocks, 1 header, 1 footer, 20.5ms
image 2554/2605 /tf/notebook/testimage/screencapture-stylelq-intro-do-2022-09-29-09_31_32.jpg: 640x192 4 banners, 2 2blocks, 1 3block, 1 4block, 1 typo, 1 header, 44.8ms
image 2555/2605 /tf/notebook/testimage/screencapture-supershop-7uptheme-net-home-4-2022-09-28-14_24_16.jpg: 640x320 2 banners, 1 2block, 1 3block, 2 4blocks, 1 typo, 3 gallerys, 1 header, 1 footer, 71.7ms
image 2556/2605 /tf/notebook/testimage/screencapture-supershop-7uptheme-net-home-5-2022-09-28-14_24_22.jpg: 640x544 3 banners, 3 4blocks, 2 5blocks, 1 typo, 1 gallery, 1 header, 1 footer, 237.5ms
image 2557/2605 /tf/notebook/testimage/screencapture-tatiana-co-kr-2022-09-29-16_51_51.jpg: 640x160 2 banners, 1 3block, 1 footer, 7.0ms
image 2558/2605 /tf/notebook/testimage/screencapture-terina-2-novaworks-net-2022-09-23-15_26_292070.jpg: 640x224 2 banner

image 2597/2605 /tf/notebook/testimage/screencapture-woodmart-xtemos-home-cars-demo-cars-2022-09-22-11_21_59.jpg: 640x384 2 banners, 1 2block, 5 3blocks, 1 4block, 1 typo, 1 gallery, 1 footer, 28.6ms
image 2598/2605 /tf/notebook/testimage/screencapture-woohoo-axiomthemes-2022-09-26-17_40_20.jpg: 640x224 2 banners, 1 2block, 2 3blocks, 2 4blocks, 1 6block, 1 header, 1 footer, 19.3ms
image 2599/2605 /tf/notebook/testimage/screencapture-wordpress-templatemela-woo-WCM02-WCM020039-WP1-2022-09-27-10_32_40.jpg: 640x256 2 banners, 4 2blocks, 3 4blocks, 1 5block, 1 header, 45.3ms
image 2600/2605 /tf/notebook/testimage/screencapture-wordpress-vinagecko-net-t-dorado-home-1-2022-09-27-09_45_57.jpg: 640x256 1 banner, 1 2block, 4 3blocks, 3 4blocks, 1 6block, 1 typo, 1 header, 1 footer, 199.5ms
image 2601/2605 /tf/notebook/testimage/screencapture-wordpress-vinagecko-net-t-matalo-2022-09-23-16_21_06.jpg: 640x384 1 banner, 1 2block, 1 3block, 4 4blocks, 1 5block, 1 gallery, 1 header, 2 footers, 6.6ms


# Test add_learning

In [ ]:
yaml_text = """
train: /tf/notebook/datasets/step1_addL/train
val: /tf/notebook/datasets/step1_addL/val
test: /tf/notebook/datasets/step1_addL/test

nc: 
names:
    ['2Block',
     '3Block',
     '4Block',
     '5Block',
     '6Block',
     '7Block',
     '8Block',
     'banner',
     'footer',
     'gallery',
     'header',
     'nblock',
     'typo']
    """

with open('/tf/notebook/datasets/step1_addL/data.yaml', 'w') as file:
    file.write(yaml_text)

# folder 별로 이미지 분류

In [ ]:
import os
import shutil

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/datasets/step1_addL/crops'

# train, test, valid 폴더 생성
train_path = '/tf/notebook/datasets/step1_addL/train'
test_path = '/tf/notebook/datasets/step1_addL/test'
valid_path = '/tf/notebook/datasets/step1_addL/val'
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# 클래스 레이블 리스트
class_labels = [
    '2block', '3block', '4block', '5block', '6block', '7block',
    '8block', 'banner', 'footer', 'gallery', 'header', 'nblock', 'typo'
]

# 클래스 레이블별로 이미지 데이터 분배
for label in class_labels:
    label_folder = os.path.join(dataset_path, label)
    image_files = [f for f in os.listdir(label_folder) if f.endswith('.jpg')]
    
    # 클래스 별 데이터를 train:test:valid = 70:15:15 비율로 분배
    train_size = int(0.8 * len(image_files))
    test_size = int(0.1 * len(image_files))
    valid_size = int(0.1 * len(image_files))
    
    # 클래스 별 train, test, valid 폴더에 클래스 레이블 폴더를 만들고 이미지 데이터를 분배
    for filename in image_files[:train_size]:
        image_path = os.path.join(label_folder, filename)
        os.makedirs(os.path.join(train_path, label), exist_ok=True)
        shutil.copy(image_path, os.path.join(train_path, label, filename))
    
    for filename in image_files[train_size:train_size+test_size]:
        image_path = os.path.join(label_folder, filename)
        os.makedirs(os.path.join(test_path, label), exist_ok=True)
        shutil.copy(image_path, os.path.join(test_path, label, filename))
    
    for filename in image_files[train_size+test_size:]:
        image_path = os.path.join(label_folder, filename)
        os.makedirs(os.path.join(valid_path, label), exist_ok=True)
        shutil.copy(image_path, os.path.join(valid_path, label, filename))

In [ ]:
from ultralytics import YOLO
import torch, gc

gc.collect()
torch.cuda.empty_cache()
model = YOLO('yolov8l.pt')

model.train(data='/tf/notebook/step2_1_imagedata/data.yaml', 
                    epochs=100,
                    batch=4,
                    imgsz=640,
                    lr0=0.01,
                    name='1221_all_class')

New https://pypi.org/project/ultralytics/8.0.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.221 🚀 Python-3.11.0rc1 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/tf/notebook/step2_1_imagedata/data.yaml, epochs=100, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=1221_all_class6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_fr

train: Scanning /tf/notebook/step2_1_imagedata/train/labels.cache... 14094 images, 1424 backgrounds, 5 corrupt: 100%|██████████| 14099/14099 [00:00<?, ?it/s]

train: WARNING ⚠️ /tf/notebook/step2_1_imagedata/train/images/2023122014423712202373.jpeg: ignoring corrupt image/label: image file is truncated (13 bytes not processed)
train: WARNING ⚠️ /tf/notebook/step2_1_imagedata/train/images/2023122015145312204724.jpeg: ignoring corrupt image/label: image file is truncated (18 bytes not processed)
train: WARNING ⚠️ /tf/notebook/step2_1_imagedata/train/images/2023122015375512201953.jpeg: ignoring corrupt image/label: image file is truncated (4 bytes not processed)
train: WARNING ⚠️ /tf/notebook/step2_1_imagedata/train/images/2023122016091912203277.jpeg: ignoring corrupt image/label: image file is truncated (2 bytes not processed)
train: WARNING ⚠️ /tf/notebook/step2_1_imagedata/train/images/2023122016233812209785.jpeg: ignoring corrupt image/label: image file is truncated (5 bytes not processed)



val: Scanning /tf/notebook/step2_1_imagedata/val/labels.cache... 1803 images, 189 backgrounds, 0 corrupt: 100%|██████████| 1803/1803 [00:00<?, ?it/s]


Plotting labels to runs/detect/1221_all_class6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/1221_all_class6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     80/100      3.81G    0.06564     0.2002       0.84          5        640: 100%|██████████| 3524/3524 [06:19<00:00,  9.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:16<00:00, 13.93it/s]

                   all       1803       1620      0.944      0.995      0.989      0.987



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     81/100      3.81G    0.06465     0.1994     0.8404          8        640:  79%|███████▉  | 2781/3524 [04:51<01:17,  9.61it/s]

In [10]:
!yolo predict detect model=/tf/notebook/runs/detect/1221_all_class4/weights/best.pt source=/tf/notebook/test/5block

Ultralytics YOLOv8.0.221 🚀 Python-3.11.0rc1 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
Model summary (fused): 268 layers, 43622799 parameters, 0 gradients, 164.9 GFLOPs

image 1/83 /tf/notebook/test/5block/LECOQGOLF.jpg: 160x512 (no detections), 74.6ms
image 2/83 /tf/notebook/test/5block/LECOQGOLF2.jpg: 160x512 (no detections), 8.0ms
image 3/83 /tf/notebook/test/5block/LECOQGOLF3.jpg: 224x512 1 HI32I11, 72.9ms
image 4/83 /tf/notebook/test/5block/LECOQGOLF4.jpg: 160x512 1 HTI41, 9.4ms
image 5/83 /tf/notebook/test/5block/LECOQGOLF5.jpg: 160x512 (no detections), 8.2ms
image 6/83 /tf/notebook/test/5block/LECOQGOLF6.jpg: 160x512 1 HTI41, 7.9ms
image 7/83 /tf/notebook/test/5block/LECOQGOLF7.jpg: 160x512 (no detections), 7.9ms
image 8/83 /tf/notebook/test/5block/LECOQSPORTIF.jpg: 320x512 1 HTI41, 71.3ms
image 9/83 /tf/notebook/test/5block/MUNSINGWEAR.jpg: 192x512 (no detections), 82.6ms
image 10/83 /tf/notebook/test/5block/MUNSINGWEAR2.jpg: 160x512 (no detections), 9.9ms
ima

# import tensorflow as tf

# GPU 사용 가능 여부 확인
if tf.test.is_gpu_available():
    print("GPU 사용 가능")
    # GPU 장치 목록 확인
    gpu_devices = tf.config.experimental.list_physical_devices('GPU')
    print("사용 가능한 GPU 장치:", gpu_devices)
else:
    print("GPU 사용 불가능")

In [ ]:
pip install -U ultralytics

In [ ]:
import os

# 비교할 두 폴더 경로 설정
folder1 = '/tf/notebook/nuggi/train/images'
folder2 = '/tf/notebook/nuggi/train/labels'

# 두 폴더 내의 파일 목록 가져오기 (확장자 포함)
files1 = [os.path.splitext(file)[0] for file in os.listdir(folder1)]
files2 = [os.path.splitext(file)[0] for file in os.listdir(folder2)]

# folder1에만 있는 파일 찾기
files_only_in_folder1 = set(files1) - set(files2)

# folder1에서 파일 삭제
for file_name in files_only_in_folder1:
    file_path = os.path.join(folder1, file_name)
    # 파일을 삭제하기 전에 백업 또는 확인 절차를 거칠 것을 권장합니다.
    os.remove(file_path)
    print(f"{file_path} 삭제 완료")

print("삭제된 파일 수:", len(files_only_in_folder1))


In [ ]:
from ultralytics import YOLO

model = YOLO('/tf/notebook/ElementDetect.pt')

image = '/tf/notebook/2.jpg'

model.predict(image, save=True, imgsz=640)

In [ ]:
model.summary

In [ ]:
!yolo predict model=/tf/notebook/ElementDetect.pt source=/tf/notebook/3.jpg

In [ ]:
from ultralytics import YOLO

model = YOLO('/tf/notebook/ElementDetect.pt')

image = '/tf/notebook/header/screencapture-blueskytechco-cogito-rtl-2022-09-27-11_39_58134.jpg'

model.predict(image, save=True, imgsz=640, conf=0.3)

In [ ]:
import os
file_paths = os.listdir('/tf/notebook/header')

for file in file_paths:
    image = os.path.join('/tf/notebook/header', file)

    model.predict(image, save=True, imgsz=640, conf = 0.5)
    